In [59]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score


In [60]:
data = pd.read_csv('noticias.csv', index_col=0)
data.head()

,title,text,label
8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [61]:
data.tail()

,title,text,label
4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL
4330,Jeb Bush Is Suddenly Attacking Trump. Here's W...,Jeb Bush Is Suddenly Attacking Trump. Here's W...,REAL


In [62]:
data['fulltext'] = data['title'].str.cat(data['text'], sep='. ')
df = data.copy()
df = df.drop(columns=['title', 'text'])
df

,label,fulltext
8476,FAKE,You Can Smell Hillary’s Fear. Daniel Greenfiel...
10294,FAKE,Watch The Exact Moment Paul Ryan Committed Pol...
3608,REAL,Kerry to go to Paris in gesture of sympathy. U...
10142,FAKE,Bernie supporters on Twitter erupt in anger ag...
875,REAL,The Battle of New York: Why This Primary Matte...
...,...,...
4490,REAL,State Department says it can't find emails fro...
8062,FAKE,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
8622,FAKE,Anti-Trump Protesters Are Tools of the Oligarc...
4021,REAL,"In Ethiopia, Obama seeks progress on peace, se..."


### Split 

In [116]:
X = df['fulltext'].tolist()
y = df['label'].tolist()


In [117]:
# countVec = CountVectorizer()
# X = countVec.fit_transform(X)
# y = countVec.fit_transform(y)

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

4751 4751
1584 1584


### CountVectorizer

In [119]:
print(y_train[0:10])
countVec = CountVectorizer()
X_train = countVec.fit_transform(X_train)
y_train = countVec.fit_transform(y_train)
X_test = countVec.fit_transform(X_test)
y_test = countVec.fit_transform(y_test)
print(y_train[0:10])

['REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL', 'FAKE', 'FAKE']
  (0, 1)	1
  (1, 0)	1
  (2, 1)	1
  (3, 1)	1
  (4, 1)	1
  (5, 0)	1
  (6, 0)	1
  (7, 1)	1
  (8, 0)	1
  (9, 0)	1


In [120]:
X_train = X_train.toarray()
y_train = y_train.toarray().flatten()
X_test = X_test.toarray()
y_test = y_test.toarray().flatten()

### Naive Bayes

In [121]:
nb = GaussianNB()

Fit

In [122]:
nb.fit(X_train, y_train)


ValueError: Expected 2D array, got 1D array instead:
array=[0 0 0 ... 0 0 0].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.